In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_raw = pd.read_csv('C:\\Users\\silri\\Applied_AI\\ML_Classification_Package\\3._Logisitic_Regression\\Facebook_Ads_2.csv',encoding='latin1')

In [3]:
df = df_raw

In [4]:
df.head()

,Names,emails,Country,Time Spent on Site,Salary,Clicked
0,Martina Avila,cubilia.Curae.Phasellus@quisaccumsanconvallis.edu,Bulgaria,25.649648,55330.06006,0
1,Harlan Barnes,eu.dolor@diam.co.uk,Belize,32.456107,79049.07674,1
2,Naomi Rodriquez,vulputate.mauris.sagittis@ametconsectetueradip...,Algeria,20.945978,41098.60826,0
3,Jade Cunningham,malesuada@dignissim.com,Cook Islands,54.039325,37143.35536,1
4,Cedric Leach,felis.ullamcorper.viverra@egetmollislectus.net,Brazil,34.249729,37355.11276,0


**The available columns**

In [5]:
df.columns

Index(['Names', 'emails', 'Country', 'Time Spent on Site', 'Salary',
       'Clicked'],
      dtype='object')

**How to get the idex of a given column**

In [6]:
df.columns.get_loc('Names')

0

- We will create two datasets X and y. X will contain all the features and y will contain only the target variable which is the class label here.

- From the dataset above the name and email will not contribute much to the model.Therefore we can drop those two columns from the dataset and take rest of the columns into account.

In [7]:
X = df.drop(columns=['Names', 'emails','Clicked'])
X

,Country,Time Spent on Site,Salary
0,Bulgaria,25.649648,55330.06006
1,Belize,32.456107,79049.07674
2,Algeria,20.945978,41098.60826
3,Cook Islands,54.039325,37143.35536
4,Brazil,34.249729,37355.11276
...,...,...,...
494,Sao Tome and Principe,19.222746,44969.13495
495,Nepal,22.665662,41686.20425
496,Zimbabwe,35.320239,23989.80864
497,Philippines,26.539170,31708.57054


In [8]:
y = df['Clicked']
y

0      0
1      1
2      0
3      1
4      0
      ..
494    0
495    0
496    0
497    0
498    1
Name: Clicked, Length: 499, dtype: int64

In [9]:
X['Country'].unique().shape

(210,)

In [10]:
## Split the dataset into two halves --> Train and Test data-set

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [13]:
## Perform OHE on top of the country column
## Perform MinMax on top of Time Spent
## Perform Standard scalling on top of Salary
## Perform Logistic regression on top of X train dataset

In [71]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import FunctionTransformer

##Use shift+M to merge the cells.

In [81]:
X

,Country,Time Spent on Site,Salary
0,Bulgaria,25.649648,55330.06006
1,Belize,32.456107,79049.07674
2,Algeria,20.945978,41098.60826
3,Cook Islands,54.039325,37143.35536
4,Brazil,34.249729,37355.11276
...,...,...,...
494,Sao Tome and Principe,19.222746,44969.13495
495,Nepal,22.665662,41686.20425
496,Zimbabwe,35.320239,23989.80864
497,Philippines,26.539170,31708.57054


In [82]:
X_train.sample(5)

,Country,Time Spent on Site,Salary
118,Israel,40.488178,63514.82268
106,Togo,25.910277,20.00000
196,Madagascar,35.496891,65822.94645
424,Tokelau,43.186689,90494.10636
201,Uganda,20.581887,26876.97102


#### Use of Column Transformers

In [59]:
# oh = OneHotEncoder(drop='first',sparse=False)
# oh.fit_transform(X_train).shape

In [89]:
trf1 = ColumnTransformer([(    
    'MinMax_timeSpent',
    MinMaxScaler(),
    #[X.columns.get_loc('Time Spent on Site')]
    [1]

    )
]
    
    , 
                         remainder = 'passthrough')

In [134]:
trf2 = ColumnTransformer([(    
                            'StandardScaling_Salary',
                            StandardScaler(),
                            #[df.columns.get_loc('Salary')]
                            [2])
                        ], remainder = 'passthrough')

#### Use of Function transformers

In [111]:
def ndarray_to_dataFrame(ndarray):
    df = pd.DataFrame(ndarray, columns=['timespent', 'salary', 'country'])
    return df

In [116]:
trf3 = FunctionTransformer(ndarray_to_dataFrame, validate=False)

In [126]:
def oneHotEncoding(df):
    column ='country'
    df_column = pd.get_dummies(df[column])
    df.drop(column, axis = 1)
    df_final = pd.concat([df,df_column], axis = 1)
    return df_final   

In [129]:
trf4 = FunctionTransformer(oneHotEncoding, validate=False)

#### Use of Sklearn pipelines

In [130]:
pipe1 = Pipeline(memory=None,
               steps =[
                   
                   ('MinMax Scaling on Time Spent',trf1),
                   ('Standard Scaling on Salary',trf2),
                   ('Convert Back to Df', trf3),
                   ('One Hot Encoding of the countries', trf4)
               ]
               ,               
               verbose=True)

In [133]:
pipe1.fit_transform(X)

[Pipeline]  (step 1 of 4) Processing MinMax Scaling on Time Spent, total=   0.0s
[Pipeline]  (step 2 of 4) Processing Standard Scaling on Salary, total=   0.0s
[Pipeline]  (step 3 of 4) Processing Convert Back to Df, total=   0.0s
[Pipeline]  (step 4 of 4) Processing One Hot Encoding of the countries, total=   0.0s


,timespent,salary,country,Afghanistan,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, United States",Wallis and Futuna,Western Sahara,Yemen,Zimbabwe,Åland Islands
0,0.128258,0.375448,Bulgaria,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.378592,0.499202,Belize,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.621945,0.289927,Algeria,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.830444,0.891624,Cook Islands,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.819281,0.531813,Brazil,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,-0.417912,0.258595,Sao Tome and Principe,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,-0.59097,0.321194,Nepal,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,-1.523825,0.551277,Zimbabwe,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
497,-1.116935,0.391621,Philippines,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
